In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import json
import requests
import os
import pandas as pd
import pandas_datareader.data as web
import datetime as dt

import bs4 as bs

from tqdm import tqdm_notebook
from typing import List
from typing import Optional
# Notebook rubbish
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [35]:
from constants import START
from constants import END
from constants import STK_DIR
from constants import SNP_TICKERS_PATH
from constants import PRICE_HIST_PATH

In [37]:
from shared import df_from_csv
from shared import load_json

### Save and load S&P tickers

In [18]:
def load_snp500_tickers(path: str, force: bool) -> List[str]:
    if os.path.exists(path) and not force:
        return load_json(path)
    resp = requests.request(url='https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', method='get')
    soup = bs.BeautifulSoup(resp.text)
    table = soup.find('table', {'class': "wikitable sortable"})
    tickers = sorted(
        # First column is the ticker column
        row.findAll('td')[0].text.strip()
        # Remove header row
        for row in table.findAll('tr')[1:]
    )
    with open(path, 'w') as outfile:
        json.dump(fp=outfile, obj=tickers)
    return tickers

### Download price history of stocks in S&P500

In [ ]:
def save_tickers_from_yahoo(tickers: List[str], stock_dir: str, force: str, start_dt: dt.date, end_dt: dt.date) -> None:
    if not os.path.exists(stock_dir):
        os.makedirs(stock_dir)
    skipped = []
    for t in tqdm_notebook(tickers):
        maybe_df = save_single_ticker_from_yahoo(
            ticker=t,
            path=os.path.join(stock_dir, f'{t}.csv'),
            force=force,
            start_dt=start_dt,
            end_dt=end_dt,
        )
        if maybe_df is None:
            skipped.append(t)
    print(f'Skipped stocks as they already exist: {skipped}')


def save_single_ticker_from_yahoo(ticker: str, path: str, force: str, start_dt: dt.date, end_dt: dt.date) -> Optional[pd.DataFrame]:
    if os.path.exists(path) and not force:
        return
    try:
        df = web.DataReader(
            name=ticker,
            data_source='yahoo',
            start=start_dt,
            end=end_dt,
        )
        df.to_csv(path)
    except Exception as e:
        print(ValueError(f'{ticker} failed to download, skipping'))
    else:
        return df

In [12]:
tickers = load_snp500_tickers(path=SNP_TICKERS_PATH, force=False)

In [19]:
save_tickers_from_yahoo(
    tickers=tickers,
    stock_dir=STK_DIR,
    force=False,
    start_dt=START,
    end_dt=END,
)

/Users/visp/venvs/venv37/lib/python3.7/site-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


BF.B failed to download, skipping
BRK.B failed to download, skipping

Skipped stocks as they already exist: ['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADS', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AGN', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF.B', 'BRK.B']


### Load saved stock data and combine dfs

In [25]:
def combine_price_history(tickers: List[str], stock_dir: str) -> pd.DataFrame:
    full_df = pd.DataFrame()
    for t in tqdm_notebook(tickers):
        try:
            df = df_from_csv(path=os.path.join(stock_dir, f'{t}.csv'))
        except Exception as e:
            print(ValueError(f'Missing ticker {t}...'))
        else:
            df = df[['Adj Close']]
            df.columns = [t]
            if full_df.empty:
                full_df = df
            else:
                full_df = full_df.join(df, how='outer')
    return full_df

In [26]:
df = combine_price_history(tickers=tickers, stock_dir=STK_DIR)

/Users/visp/venvs/venv37/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Missing ticker BF.B...
Missing ticker BRK.B...



In [30]:
display(df.shape)
display(df.head())
df.info()

(5100, 501)

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,44.294170,NaN,NaN,3.470226,NaN,2.995721,18.25000,6.592210,NaN,16.274673,...,33.437664,21.549223,6.954667,51.922791,NaN,4.402331,NaN,25.027779,41.202934,NaN
2000-01-04,40.910591,NaN,NaN,3.177650,NaN,2.791193,17.81250,6.403859,NaN,14.909401,...,32.677715,21.136469,6.954667,49.507771,NaN,4.313842,NaN,24.666666,39.207741,NaN
2000-01-05,38.372894,NaN,NaN,3.224152,NaN,3.007750,18.00000,6.392088,NaN,15.204173,...,31.962463,22.288738,7.074894,52.056961,NaN,4.335964,NaN,25.138889,39.161343,NaN
2000-01-06,36.911816,NaN,NaN,2.945139,NaN,3.236340,18.03125,6.615753,NaN,15.328290,...,29.190891,23.441015,7.084142,50.983620,NaN,4.299095,NaN,23.777779,39.718132,NaN
2000-01-07,39.987797,NaN,NaN,3.084645,NaN,3.657427,17.93750,6.686383,NaN,16.072983,...,32.498898,23.372227,7.065648,52.191128,NaN,4.203231,NaN,23.513889,39.810936,NaN


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5100 entries, 2000-01-03 to 2020-04-09
Columns: 501 entries, A to ZTS
dtypes: float64(501)
memory usage: 19.5 MB


In [33]:
df.to_csv(PRICE_HIST_PATH)